In [1]:
import os
import sys
import json
filename = 'selected.jsonl'
with open(filename, 'r', encoding='utf-8') as file:
    story_data = []
    for line in file:
        data = json.loads(line.strip())
        story_data.append(data)
stories = {}
story_id_list = []
for story_da in story_data:
    story_id_list.append(story_da['storyID'])
    stories[story_da['storyID']] = story_da['story']

In [2]:
# import spacy

# nlp = spacy.load("en_core_web_sm")
# def get_character(sentences):
#     character = []
#     for sent in sentences:
#         doc = nlp(sent)
#         for ent in doc.ents:
#             if ent.label_ == "PERSON":
#                 character.append(ent.text)
#     # 去重
#     character = list(set(character))
#     return character

In [3]:
import stanza
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
def get_character(sentences):
    text = " ".join(sentences)
    doc = nlp(text)
    names = []

    for sentence in doc.sentences:
        for entity in sentence.ents:
            if entity.type == 'PERSON':
                name = entity.text.split("'")[0]
                names.append(name)
    character = list(set(names))
    return character

/home/hansirui/Briding_incoherence/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-15 11:37:45 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-06-15 11:37:46 INFO: Downloaded file to /home/hansirui/stanza_resources/resources.json
2024-06-15 11:37:46 WARNING: Language en package default expects mwt, which has been added
2024-06-15 11:37:46 INFO: Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| ner       | ontonotes-ww-multi_charlm |

2024-06-15 11:37:47 INFO: Using device: cu

In [4]:
character_table = {}
for story_id in story_id_list:
    sentences = stories[story_id]
    character = get_character(sentences)
    character_table[story_id] = character

In [5]:
# character_table
# save to json file
with open('character_table.json', 'w', encoding='utf-8') as file:
    json.dump(character_table, file, ensure_ascii=False, indent=4)

In [6]:
# 判断是否有重叠
def compare_character(story_id1, story_id2):
    character1 = character_table[story_id1]
    character2 = character_table[story_id2]
    for c in character1:
        if c in character2:
            return True

import random
# zip the storyid and the sentence(and wrong ending) the shffule the list
def shuffle_list(l):
    random.seed(42) 
    B_l = l.copy()
    C_l = l.copy()
    while True:
        random.shuffle(B_l)
        random.shuffle(C_l)
        for a, b, c in zip(l, B_l, C_l):
            if a == b or compare_character(a, b):
                break
            elif a == c or compare_character(a, c):
                break
            elif b == c or compare_character(b, c):
                break
        else:
            return B_l, C_l

In [7]:
B_story_id_list, C_story_id_list = shuffle_list(story_id_list)

In [8]:
def filling_template(story):
    whole_plot_points = ""
    for i, plot_point in enumerate(story):
        whole_plot_points += f"{i + 1}. {plot_point}\n"
    # prompt = f"""Craft a concise and coherent short story that meticulously incorporates each of the {len(story)} plot points provided below, ensuring absolute fidelity to the given details. Additional details may be included only when necessary to maintain the coherence of the narrative.\n\n### Plot Points\n{whole_plot_points}\nPlease output the story without any further instructions or explanations."""

    # prompt = f"""Write a concise and coherent short story that weaves all {len(story)} plot points provided below, ensuring absolute fidelity to the given details. Additional details may be included only when necessary to maintain the coherence of the narrative.\n\n### Plot Points\n{whole_plot_points}\nYou only need to output the story."""
    COT_prompt = f"""Write a concise and coherent short story that weaves all {len(story)} plot points provided below, ensuring absolute fidelity to the given details. Additional details may be included only when necessary to maintain the coherence of the narrative.\n\n### Plot Points\n{whole_plot_points}\nMake a plan then write. Your output should be of the following format:\n\nPlan:\nYour plan here.\n\nStory:\nYour story here."""
    return COT_prompt

import random
def get_outliine(j, type):
    id_A = story_id_list[j]
    id_B = B_story_id_list[j]
    id_C = C_story_id_list[j]


    story = []

    for i in range(len(type) - 1):
        ch1 = type[i]
        ch2 = type[i + 1]
        
        if ch1 == "A" or ch1 == "a":
            point = stories[id_A][int(ch2)]
            story.append(point)
        elif ch1 == "B" or ch1 == "b":
            point = stories[id_B][int(ch2)]
            story.append(point)
        elif ch1 == "C" or ch1 == "c":
            point = stories[id_C][int(ch2)]
            story.append(point)
        else:
            continue
    return story, filling_template(story)

In [9]:
types =["a0 a1 B3 a3 a4", "a0 B0 a2 B4 a4","a0 B0 C0 a2 B4 C4 a4","a0 B0 a1 B1 a2 B2 a3 B3 a4 B4"]
type_name = ["主线情节点", "主线支线", "主线双支线", "双主线"]
for i in range(4):
    print(f"第{i+1}个类型：{type_name[i]}")
    filename = f'task_story/COT/{type_name[i]}.jsonl'
    type = types[i]

    with open(filename, 'w') as f:
        for j in range(len(story_id_list)):
            outline, prompt = get_outliine(j, type)
            if 'C' in type:
                json_record = {'storyID_A': story_id_list[j], 'storyID_B': B_story_id_list[j], 'storyID_C': C_story_id_list[j], 'prompt':prompt ,'outline':outline}
            else:
                json_record = {'storyID_A': story_id_list[j], 'storyID_B': B_story_id_list[j], 'prompt':prompt, 'outline':outline}
            f.write(json.dumps(json_record) + '\n')

第1个类型：主线情节点
第2个类型：主线支线
第3个类型：主线双支线
第4个类型：双主线


## test

#### 主线+情节点

In [10]:
prompt  = get_outliine(121, "a0 a1 B3 a3 a4")
print(prompt)

(['Jason woke up to the sound of water.', "It was coming from Jason's basement.", 'When Chris tried it Chris was amazed at how bright it was outside.', 'The basement was totally flooded from a leak!', 'Jason called a plumber to fix everything.'], "Write a concise and coherent short story that weaves all 5 plot points provided below, ensuring absolute fidelity to the given details. Additional details may be included only when necessary to maintain the coherence of the narrative.\n\n### Plot Points\n1. Jason woke up to the sound of water.\n2. It was coming from Jason's basement.\n3. When Chris tried it Chris was amazed at how bright it was outside.\n4. The basement was totally flooded from a leak!\n5. Jason called a plumber to fix everything.\n\nMake a plan then write. Your output should be of the following format:\n\nPlan:\nYour plan here.\n\nStory:\nYour story here.")


### 主线+支线


In [11]:
outline,prompt = get_outliine(121, "a0 B0 a2 B4 a4")
print(prompt)

Write a concise and coherent short story that weaves all 5 plot points provided below, ensuring absolute fidelity to the given details. Additional details may be included only when necessary to maintain the coherence of the narrative.

### Plot Points
1. Jason woke up to the sound of water.
2. Chris had been agoraphobic for years.
3. Jason went down to the basement to see what was happening.
4. Chris promised his worker he would try it more.
5. Jason called a plumber to fix everything.

Make a plan then write. Your output should be of the following format:

Plan:
Your plan here.

Story:
Your story here.


### 主线+2支线

In [12]:
outline,prompt  = get_outliine(121, "a0 B0 C0 a2 B4 C4 a4")
print(prompt)

Write a concise and coherent short story that weaves all 7 plot points provided below, ensuring absolute fidelity to the given details. Additional details may be included only when necessary to maintain the coherence of the narrative.

### Plot Points
1. Jason woke up to the sound of water.
2. Chris had been agoraphobic for years.
3. There was a clown at Eddy's friend's birthday party.
4. Jason went down to the basement to see what was happening.
5. Chris promised his worker he would try it more.
6. Eddy's mom told Eddy that they wore big shoes, but they didn't have extra big feet.
7. Jason called a plumber to fix everything.

Make a plan then write. Your output should be of the following format:

Plan:
Your plan here.

Story:
Your story here.


### 双主线

In [13]:
prompt  = get_outliine(12, "a0 B0 a1 B1 a2 B2 a3 B3 a4 B4")
print(prompt)

(["Ruth's neighbor was religious.", 'Maggie had been in trouble with the law when she was younger.', "Ruth's neighbor invited Ruth over for a bible study.", 'When Maggie married and had a kid, Maggie was a great mom.', 'Ruth reluctantly agreed.', 'Maggie and her husband divorced and her husband filed for custody.', 'Ruth and her neighbor read several passages and discussed them.', "The judge held Maggie's past against her and ruled for her husband.", 'When Ruth and her neighbor were done, Ruth went home.', 'Maggie, despondent, fell back into a life of crime.'], "Write a concise and coherent short story that weaves all 10 plot points provided below, ensuring absolute fidelity to the given details. Additional details may be included only when necessary to maintain the coherence of the narrative.\n\n### Plot Points\n1. Ruth's neighbor was religious.\n2. Maggie had been in trouble with the law when she was younger.\n3. Ruth's neighbor invited Ruth over for a bible study.\n4. When Maggie ma